In [1]:
import re 
import time
import selenium
import requests
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urlparse
from selenium.common.exceptions import TimeoutException

The website does not contain any revealing anchor tags, sitemap.xml had to be used to extract the hidden urls. Contents as such ratings, were hidden as well. 

In [2]:
def get_all_urls(url):

    xml = get_sitemap(url)
    sitemap_type = get_sitemap_type(xml)

    if sitemap_type =='sitemapindex':
        sitemaps = get_child_sitemaps(xml)
    else:
        sitemaps = [url]
    df = pd.DataFrame(columns=['loc'])

    for sitemap in sitemaps:
        sitemap_xml = get_sitemap(sitemap)
        df_sitemap = sitemap_to_dataframe(sitemap_xml, name=sitemap)

        df = pd.concat([df, df_sitemap], ignore_index=True)

    return df

def get_sitemap(url):
    response = urllib.request.urlopen(url)
    xml = BeautifulSoup(response, 
                         'lxml-xml', 
                         from_encoding=response.info().get_param('charset'))
    return xml

def get_sitemap_type(xml):
    sitemapindex = xml.find_all('sitemapindex')
    sitemap = xml.find_all('urlset')

    if sitemapindex:
        return 'sitemapindex'
    elif sitemap:
        return 'urlset'
    else:
        return

def sitemap_to_dataframe(xml, name=None, data=None, verbose=False):
    df = pd.DataFrame(columns=['loc'])

    urls = xml.find_all("url")
  
    for url in urls:

        if xml.find("loc"):
            loc = url.findNext("loc").text
            parsed_uri = urlparse(loc)
            domain = '{uri.netloc}'.format(uri=parsed_uri)
        else:
            loc = ''
            domain = ''

        if name:
            sitemap_name = name
        else:
            sitemap_name = ''
              
        row = {
            'loc': loc,
        }

        if verbose:
            print(row)

        df = df.append(row, ignore_index=True)
    return df

url = "https://www.senheng.com.my/sitemap.xml"
xml = get_sitemap(url)
df_URL = get_all_urls(url)
df_URL.to_csv('URL.csv')

In [3]:
df1 = df_URL.copy()
df1['sitemap_cat'] = df1['loc'].str.split('/').str[3]
df1.dropna()

,loc,sitemap_cat
1,https://www.senheng.com.my/all-products,all-products
2,https://www.senheng.com.my/all-products/digita...,all-products
3,https://www.senheng.com.my/digital-gadgets/cam...,digital-gadgets
4,https://www.senheng.com.my/digital-gadgets/drones,digital-gadgets
5,https://www.senheng.com.my/digital-gadgets/wea...,digital-gadgets
...,...,...
2953,https://www.senheng.com.my/redmi-note-11-pro-5...,redmi-note-11-pro-5g-8gb+128gb
2954,https://www.senheng.com.my/bosch-ideal-smart-l...,bosch-ideal-smart-lock-id60p
2955,https://www.senheng.com.my/russell-taylors-600...,russell-taylors-600w-hand-blender-hb-6
2956,https://www.senheng.com.my/oppo-a74-a76-6gb-128gb,oppo-a74-a76-6gb-128gb


In [4]:
df1.to_csv('sitemap.csv')

In [5]:
# Calling Chrome Webdriver
driver=webdriver.Chrome("C:\Program Files (x86)\chromedriver.exe")
# driver.implicitly_wait(20)
# Open webpage
driver.get('https://www.senheng.com.my/all-products/computers-laptops.html')

In [6]:
product_name = driver.find_elements_by_xpath('//div[@class="jss117"]/a/p')
product_price = driver.find_elements_by_xpath('//div[@class="desc-item__price"]/span[1]')

In [7]:
df2 = pd.DataFrame(columns=['Product_Name','Price'])
for i in range(len(product_name)):
    df2 = df2.append({'Product_Name': product_name[i].text, 'Price': product_price[i].text},   ignore_index=True)

In [8]:
df2.to_csv('namelist.csv')

In [9]:
df4 = pd.read_csv('sitemap.csv',index_col=0)

In [10]:
sitemap = df4.loc[df4['sitemap_cat'].str.contains("acer|apple-imac-24-inch|apple-usb-C-digital-AV|apple-usb-C-to-Usb|apple-usb-C-vga|apple-macbook-air|apple-macbook-pro|apple-magic-keyboard-for-ipad-air|apple-magic-keyboard-with|apple-magic-trackpad|Microsoft|Asus|boomwave|canon|dynabook|epson|honor-MagicBook|hp-14S|hp-245G8|hp-680|hp-682|hp-All|hp-deskjet|hp-envy|hp-laptop|hp-pavilion|hp-M24F|hp-slim|hp-spectre|hp-victus|huawei-28.2|huawei-matebook|huawei-wifi|huawei-mateview|huawei-matebook-d15-2021|Logitech-B175|Logitech-k400|logitech-m100r|logitech-m171|logitech-m331|logitech-m377|logitech-m590|logitech-mk200|logitech-mk220|logitech-mk240|logitech-mk295|logitech-mx|logitech-wireless|mi-34|msi|promate-cooling|promate-deskmate|promate-laptop|rapoo-optical|rog-zephyrus-duo|samsung-24-Inch-Smart-Monitor|samsung-27-Inch-Smart-Monitor|samsung-32-Inch-Smart-Monitor|samsung-32GB-Evo|samsung-43-Inch-Smart-Monitor|samsung-64GB-EVO|SanDisk|targus|TP-Link-300|tp-link-4g|tp-link-5|tp-link-8|tp-link-AC1200|tp-link-ac1750|tp-link-ac2200|tp-link-ac600|tp-link-ac750|tp-link-archer|tp-link-ax|tp-link-deco|tp-link-td|tp-link-tl|tp-link-bluetooth", flags=re.IGNORECASE, na=False)]

In [11]:
sitemap

,loc,sitemap_cat
120,https://www.senheng.com.my/canon-pixma-printer...,canon-pixma-printer-cnn-e510
179,https://www.senheng.com.my/canon-laser-shot-cn...,canon-laser-shot-cnn-lbp6030
309,https://www.senheng.com.my/sandisk-ultra-flair...,sandisk-ultra-flair-usb-3-0-flash-drive-32gb-s...
316,https://www.senheng.com.my/canon-pixma-all-in-...,canon-pixma-all-in-one-printer-cnn-e410
324,https://www.senheng.com.my/sandisk-ultra-dual-...,sandisk-ultra-dual-drive-usb-type-c-32gb-sd-dd...
...,...,...
2898,https://www.senheng.com.my/microsoft-surface-p...,microsoft-surface-pro-7-plus
2899,https://www.senheng.com.my/hp-m24f-23-8-fhd-ip...,hp-m24f-23-8-fhd-ips-monitor
2912,https://www.senheng.com.my/microsoft-surface-l...,microsoft-surface-laptop-studio
2931,https://www.senheng.com.my/acer-nitro-5-ace-an...,acer-nitro-5-ace-an515-45-r7nd


In [12]:
data = sitemap.copy()
data.drop(columns='sitemap_cat')
product_links = data['loc'].tolist()
print(len(product_links))

254


In [14]:
templist=[]       
title = []
price = []
stock =[]
descr = []
rating = []
review = []

for web in product_links:
    wd = webdriver.Chrome('C:\Program Files (x86)\chromedriver.exe')
    wd.get(web)
    time.sleep(1)
    
    try:        
        title= wd.find_element_by_xpath('//div[@class="product__name-wrapper"]/h1').text
    except:     
        title= "Not available"
    try:        
        price= wd.find_element_by_xpath('//div[@class="product__name-wrapper"]/span').text
    except:        
        price= "Not available"
    try:        
        stock= wd.find_element_by_xpath('//div[@class="product__status-wrapper"]/p').text
    except:        
        stock= "Not available"
    try:        
        descr= wd.find_element_by_xpath('//div[@class="short-description__text"]/p').text
    except:        
        descr= "Not available"
    try:        
        rating= wd.find_element_by_xpath('//*[@id="maincontent"]/div[2]/div[4]/div/div/div/div/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div/button[1]/span/svg/path').text
    except:        
        rating= "Not available"           
    try:        
        review= wd.find_element_by_xpath('//div[@class="product__review-content"]/div/div[2]/p').text
    except:        
        review= "Not available"
            
    print(f"Title:- {title}")
    print(f"Price:- {price}")
    print(f"Stock:- {stock}")
    print(f"Decription:- {descr}")
    print(f"Rating:- {rating}")
    print(f"Review:- {review}")

    Table_dict = {'Title': title, 
                  'Price':price, 
                  'Stock': stock, 
                  'Descrption': descr,
                  'Rating': rating,
                  'Review': review}
    
    templist.append(Table_dict)


Title:- Canon PIXMA Printer CNN-E510
Price:- RM 309.00
Stock:- Out Of Stock
Decription:- Print, Scan & Copy
Color Inkjet Printer
Resolution up to 4,800 x 1,200 dpi
449(W) x 304(D) x 152(H)mm
5.3kg
Rating:- Not available
Review:- Not available
Title:- Canon Laser Shot CNN-LBP6030
Price:- RM 375.00
Stock:- In Stock
Decription:- High speed print jobs
Low energy consumption
Sleep mode
Auto shut down feature
Requires adapter for power cable
Rating:- Not available
Review:- Not available
Title:- SanDisk Ultra Flair USB 3.0 Flash Drive 32GB SD-CZ73-032G-G46
Price:- RM 31.00
Stock:- Out Of Stock
Decription:- Capacity: 32 GB
Read Speed: up to 150 MB/s1
Generation: USB 3.0
Rating:- Not available
Review:- Not available
Title:- Canon PIXMA All-in-One Printer CNN-E410
Price:- RM 239.00
Stock:- Out Of Stock
Decription:- Print
Scan
Copy
All-In-One Printer
426(W) x 306(D) x 145(H)mm
3.4kg
Auto Power ON
Rating:- Not available
Review:- Not available
Title:- SanDisk Ultra Dual Drive USB Type-C 32GB SD-DDC

Title:- HP DeskJet Ink Advantage 2336 All-in-One Printer
Price:- RM 215.00
Stock:- Out Of Stock
Decription:- Color
Print, copy, scan
1 Hi-Speed USB 2.0
Ink
Rating:- Not available
Review:- Not available
Title:- TP-Link Deco M5 AC1500 Whole Home Mesh Wi-Fi System (3-Pack)
Price:- RM 799.00
Stock:- In Stock
Decription:- Seamless Whole Home Wifi Coverage
Eliminate Wifi Dead Zone
Coverage Up To 5500 Sq Ft
Speed Up To 1267mbps
Come With Home Care Anti Virus
Connects Over 100 Device
Rating:- Not available
Review:- Not available
Title:- MSI 27 Inch Optix Monitor MAG272CQR
Price:- RM 1,799.00
Stock:- In Stock
Decription:- Curved Gaming Display (1500R) 
Mystic Light
WQHD High Resolution
165Hz Refresh Rate
1ms Response Time
Gaming OSD App
HDR Ready-Provide Eye-Catching Detail Through Contrast And Shadows Adjustment.
Night Vision
Connect With Type-C
AMD Freesync
Anti-Flicker And Less Blue Light 
Frameless Design
178° Wide View Angle.
Rating:- Not available
Review:- Not available
Title:- MSI Mice G

Title:- TP-Link AC2200 Smart Home Mesh Wi-Fi System DecoM9 Plus(3-pack)
Price:- RM 1,359.00
Stock:- In Stock
Decription:- Not available
Rating:- Not available
Review:- Not available
Title:- TP-Link Deco M9 Plus AC2200 Smart Home Mesh Wi-Fi System (2-Pack)
Price:- RM 999.00
Stock:- In Stock
Decription:- AC2200 Tri-Band Smart Home Mesh Wi-Fi System
IoT Hub(Bluetooth 4.2, ZigBee HA 1.2)
Qualcomm 717MHz Quad-core CPU
867Mbps at 5GHz+867Mbps at 5GHz_Backhaul+400Mbps at 2.4GHz
512MB RAM, 4GB FLASH
2 Gigabit Ports, 1 USB 2.0 Port
8 antennas
256QAM
MU-MIMO, Beamforming, DFS for Europe version, HomeCare(Parental Controls, Built-in Antivirus, Quality of Service, Reporting), Access Point Mode, IPv6 Ready, Assisted Setup, Deco App,Bluetooth Onboarding, Cloud Support
Rating:- Not available
Review:- Not available
Title:- TP-Link Deco E4 AC1200 Whole Home Mesh Wi-Fi System (3-Pack)
Price:- RM 439.00
Stock:- In Stock
Decription:- AC1200 Whole-Home Mesh Wi-Fi System
Qualcomm CPU
867Mbps at 5GHz+300Mbps

Title:- TP-Link 300Mbps LTE-Advanced Mobile Wi-Fi M7450
Price:- RM 439.00
Stock:- Out Of Stock
Decription:- Cat6 300Mbps 4G LTE-Advanced Mobile Wi-Fi
Qualcomm
LTE-FDD/LTE-TDD/DC-HSPA+/HSPA/UMTS/EDGE/GPRS/GSM
AC1200 selectable Dual Band Wi-Fi
Internal 4G Modem
SIM card slot, micro SD card slot, 1.4 inch TFT color screen display, 3000mAH rechargeable battery, tpMiFi App
Rating:- Not available
Review:- Not available
Title:- TP-Link 4G LTE Mobile Wi-Fi M7350
Price:- RM 249.00
Stock:- Out Of Stock
Decription:- 150Mbps 4G LTE Mobile Wi-Fi
Qualcomm
LTE-FDD/DC-HSPA+/HSPA/UMTS/EDGE/GPRS/GSM
N300 Single Band Wi-Fi
Internal 4G Modem
1.4 inch TFT color screen display, 2000mAh rechargeable battery, tpMiFi App
Rating:- Not available
Review:- Not available
Title:- TP-Link 4G LTE Mobile Wi-Fi M7200
Price:- RM 219.00
Stock:- Out Of Stock
Decription:- 150Mbps 4G LTE Mobile Wi-Fi
Qualcomm
LTE-FDD/LTE-TDD/DC-HSPA+/HSPA/UMTS
N300 Wi-Fi
Internal 4G Modem, 2000mAh rechargeable battery, tpMiFi App
Rating:- No

Title:- Microsoft Comfort Mouse 4500
Price:- RM 102.00
Stock:- In Stock
Decription:- Wired Mouse
Five customisable buttons
BlueTrack Technology®
Rating:- Not available
Review:- Not available
Title:- HP 14S(AMD 3020E/4GB/256GB SSD) 14-SFQ0058AU
Price:- RM 1,649.00
Stock:- Out Of Stock
Decription:- AMD 3020e
14"
4GB Ram & 256GB SSD
Windows 10 Home 64
Rating:- Not available
Review:- Not available
Title:- HP ENVY13 (i7-1165G7/16GB DDR4/512GB SSD/ MX450 2GB) 13-BA1011TX
Price:- RM 5,499.00
Stock:- Out Of Stock
Decription:- 13.3-inch
Intel Core i7-1165G7
16GB RAM / 512GB SSD
Windows 10 Home 64
Rating:- Not available
Review:- Not available
Title:- Microsoft Bluetooth Desktop-Black
Price:- RM 275.00
Stock:- In Stock
Decription:- Keyboard + Mouse
Bluetooth
2.4 GHz frequency range
Rating:- Not available
Review:- Not available
Title:- Microsoft Wireless Desktop 3050
Price:- RM 229.00
Stock:- In Stock
Decription:- Built in palm rest
Snap-in mini-transceiver
Customizable shortcut keys
Rating:- Not 

Title:- Mi 34-Inch Curved Gaming Monitor XMI-GMONITOR34
Price:- RM 2,199.00
Stock:- Out Of Stock
Decription:- 21:9 UltraWide Screen
WQHD 3440 x 1440 Resolution
1500R Extreme Curvature 121% sRGB Wid
144Hz High Refresh Rate
FreeSync Premium Variable Refresh Rate
Low Blue Light, No Visible Flicker* 
121% sRGB Wide Color Gamut
Rating:- Not available
Review:- Not available
Title:- Apple iMac 24-inch (4-ports)
Price:- RM 6,449.00 - RM 7,249.00
Stock:- In Stock
Decription:- • Immersive 24-inch 4.5K Retina display with P3 wide color gamut and 500 nits of brightness¹
• Apple M1 chip delivers powerful performance with 8-core CPU and 7- or 8-core GPU
• Strikingly thin 11.5 mm design in vibrant colors
• 1080p FaceTime HD camera with M1 ISP for amazing video quality
• Studio-quality three-mic array for crystal clear calls and voice recordings
• Six-speaker sound system for a remarkably robust and high-quality audio experience
• Up to 512GB of ultrafast SSD storage
• Two Thunderbolt / USB 4 ports an

Title:- Microsoft 365 Family (Digital Download Delivery)
Price:- RM 259.00
Stock:- In Stock
Decription:- THIS PRODUCT IS NON-REFUNDABLE.
Rating:- Not available
Review:- Not available
Title:- Microsoft 365 Personal (Digital Download Delivery)
Price:- RM 199.00
Stock:- In Stock
Decription:- THIS PRODUCT IS NON-REFUNDABLE.
Rating:- Not available
Review:- Not available
Title:- Microsoft 365 Business Standard (Digital Download Delivery)
Price:- RM 629.00
Stock:- In Stock
Decription:- THIS PRODUCT IS NON-REFUNDABLE.
Rating:- Not available
Review:- Not available
Title:- HP 245G8 R3-3300U 14 4GB/256 PC
Price:- RM 2,099.00
Stock:- Out Of Stock
Decription:- 14-inch
Rating:- Not available
Review:- Not available
Title:- ASUS Zenbook 14 [I7-1165G7/8GB+512GB/Intel Iris Xᵉ Graphics]
Price:- RM 4,699.00
Stock:- Out Of Stock
Decription:- Windows 10 Home - ASUS recommends Windows 10 Pro for business
Rating:- Not available
Review:- Not available
Title:- Microsoft Surface Go 3
Price:- RM 1,888.00 - RM 2,9

Title:- MSI GF63 Thin 11UC-236MY (Tiger Lake i5-11400H/RTX3050 Max-Q, 4GB+512GB)
Price:- RM 4,299.00
Stock:- In Stock
Decription:- Windows 11 Home
Rating:- Not available
Review:- Not available
Title:- MSI Modern 14 B11MOU-870MY (Tiger Lake i3-1115G4 / 4GB+512GB / UHD Graphics)
Price:- RM 2,699.00
Stock:- In Stock
Decription:- Windows 11 Home
Rating:- Not available
Review:- Not available
Title:- Microsoft Visio Professional 2021 (Digital Download Delivery)
Price:- RM 3,199.00
Stock:- In Stock
Decription:- THIS PRODUCT IS NON-REFUNDABLE.
Rating:- Not available
Review:- Not available
Title:- Microsoft Visio Standard 2021 (Digital Download Delivery)
Price:- RM 1,599.00
Stock:- In Stock
Decription:- THIS PRODUCT IS NON-REFUNDABLE.
Rating:- Not available
Review:- Not available
Title:- Microsoft Project Standard 2021 (Digital Download Delivery)
Price:- RM 3,199.00
Stock:- In Stock
Decription:- THIS PRODUCT IS NON-REFUNDABLE.
Rating:- Not available
Review:- Not available
Title:- Microsoft Proj

Title:- TP-Link Deco X90 AX6600 Whole Home Mesh Wi-Fi System (2-Pack)
Price:- RM 1,499.00
Stock:- In Stock
Decription:- AX6600 Tri-Band WiFi
Rating:- Not available
Review:- Not available
Title:- Acer Aspire XC Series XC340-3250W10 [AMD Athlon Gold 3150U/4GB+256GB/AMD Radeon Graphics/W11/Wireless KBM/OPI]
Price:- RM 1,749.00
Stock:- Out Of Stock
Decription:- Not available
Rating:- Not available
Review:- Not available
Title:- Microsoft Surface Pro 7+
Price:- RM 3,599.00 - RM 4,299.00
Stock:- In Stock
Decription:- A brilliant 12.3” display
Rating:- Not available
Review:- Not available
Title:- HP M24F 23.8" FHD IPS Monitor
Price:- RM 629.00
Stock:- In Stock
Decription:- 23.8"
Rating:- Not available
Review:- Not available
Title:- [Pre-order] Microsoft Surface Laptop Studio (ETA: 8 March onwards)
Price:- RM 7,399.00 - RM 9,399.00
Stock:- In Stock
Decription:- 14.4” 120Hz touchscreen
Rating:- Not available
Review:- Not available
Title:- Acer Nitro 5 ACE-AN515-45-R7ND [AMD Ryzen 5 5600H / 8GB+

WebDriverException: Message: chrome not reachable
  (Session info: chrome=98.0.4758.102)


In [15]:
df_scraped = pd.DataFrame(templist)
df_scraped.head(3)

,Title,Price,Stock,Descrption,Rating,Review
0,Canon PIXMA Printer CNN-E510,RM 309.00,Out Of Stock,"Print, Scan & Copy\nColor Inkjet Printer\nReso...",Not available,Not available
1,Canon Laser Shot CNN-LBP6030,RM 375.00,In Stock,High speed print jobs\nLow energy consumption\...,Not available,Not available
2,SanDisk Ultra Flair USB 3.0 Flash Drive 32GB S...,RM 31.00,Out Of Stock,Capacity: 32 GB\nRead Speed: up to 150 MB/s1\n...,Not available,Not available


In [16]:
def get_brand(product):
    brand = product.split()[0]
    return brand

In [17]:
df_scraped['Brand'] = df_scraped['Title'].apply(get_brand)

In [18]:
df_scraped.head(5)

,Title,Price,Stock,Descrption,Rating,Review,Brand
0,Canon PIXMA Printer CNN-E510,RM 309.00,Out Of Stock,"Print, Scan & Copy\nColor Inkjet Printer\nReso...",Not available,Not available,Canon
1,Canon Laser Shot CNN-LBP6030,RM 375.00,In Stock,High speed print jobs\nLow energy consumption\...,Not available,Not available,Canon
2,SanDisk Ultra Flair USB 3.0 Flash Drive 32GB S...,RM 31.00,Out Of Stock,Capacity: 32 GB\nRead Speed: up to 150 MB/s1\n...,Not available,Not available,SanDisk
3,Canon PIXMA All-in-One Printer CNN-E410,RM 239.00,Out Of Stock,Print\nScan\nCopy\nAll-In-One Printer\n426(W) ...,Not available,Not available,Canon
4,SanDisk Ultra Dual Drive USB Type-C 32GB SD-DD...,RM 46.00,Out Of Stock,Capacity: 32 GB\nRead Speed: up to 150 MB/s\nI...,Not available,Not available,SanDisk


In [19]:
df_scraped.to_csv('SH_Product_Data.csv', index=False)